In [1]:
from T13BFLSimpCore import *

/home/cl/Documents/n-bafc/blockchain-afc/taxi


In [2]:
dat = 'trips_simpler.csv'
company = [5,4]#[2,3,4,5]
location = [4]#[mk for mk in range(9)]

blockchain = Blockchain()

list index out of range|Brand new blockchain, creating genesis block
BLK: C:0x7fd77#  0 <-- PH=0    :PF=0    , RL={}   , #TRXS=0
Blockchain created with ID: 0x7fd77556c580 140563363317120


In [3]:
print("1 - DISTRIBUTION =================================")
import pickle
try:
    locdump = "./models/clis" + str(company) + str(location) + ".p"
    clients = pickle.load(open(locdump, "rb"))
    print("Auto-load from cache")
except:
    print("Generating dist")
    db = Distributor(dat, company, location, settings)
    db.divBoth(settings)
    clients = db.extractClients()
    import pickle
    pickle.dump(clients, open(locdump, "wb"))

1 - DISTRIBUTION =================================
Auto-load from cache


In [4]:
print("2 - AUTHORIZATION =================================")
print("Using blockchain #" + blockchain.bc_id)
for c in clients:
    c.model = CNNLSTM(settings['frame_in'],settings['frame_out'], epochs=c.epoch)
    blockchain.authorizeClient(c)
    c.assignBlockchain(blockchain)
    print(c)

2 - AUTHORIZATION =================================
Using blockchain #0x7fd77556c580
CLI:5/0-18904,   12036
MIN:5/4--3008,   43917
CLI:4/0--6381,     582
CLI:4/4--1543,   23207
CLI:3/0-69564,   61603
MIN:3/4--7216,  131480


In [5]:
def train(ep=None):
    print("\n3 - TRAINING =================================")
    for c in clients:
        c.model.setData(c.inp_train,c.out_train)
        c.train(epoch=ep)
        c.submitModelAsTransaction()

def mine():
    print("\n4 - MINING =================================")
    for c in clients:
        if type(c) == Miner:
            c.mine()

def consensus():
    print("\n5 - CONSENSUS + INCENTIVIZATION ============")
    blockchain.consensus()

def aggregate():
    print("\n6 - AGGREGATION ============================", end=' ')
    for c in clients:
        mods = []
        for tx in c.blockchain.lastBlock.transactions:
            if tx.type == "MOD":
                mods.append(tx.load)
        if len(mods) == 0:
            print("No aggregation.\n")
            return
        nmod = c.aggregate(mods)
        c.replaceWeights(nmod)
    print("done.\n")

def iterate():
    train(37)
    mine()
    consensus()
    aggregate()

In [6]:
# FL iterations
iters = 3
for i in range(iters):
    print("> ITERATION #" + str(i+1))
    iterate()

> ITERATION #1

3 - TRAINING =================================
#5/0:9.89s; CLI:5/0 MOD-->CHIDX#  1
#5/4:13.58s; MIN:5/4 MOD-->CHIDX#  1
#4/0:12.50s; CLI:4/0 MOD-->CHIDX#  1
#4/4:12.45s; CLI:4/4 MOD-->CHIDX#  1
#3/0:12.86s; CLI:3/0 MOD-->CHIDX#  1
#3/4:13.93s; MIN:3/4 MOD-->CHIDX#  1

4 - MINING =================================
Start mining: 5/4
-3008 MINE APPROVED: POA
Proposal:  18904:2.864; -3008:3.085; -6381:4.109; -1543:3.901; 69564:3.327; -7216:3.04; 
PREV =  | CURR = 18904:2.864; -3008:3.085; -6381:4.109; -1543:3.901; 69564:3.327; -7216:3.04; 
-3008 MINE APPROVED AND ACCEPTED: POQ
BLK: C:0x7fd77#  1 <-- PH=0    :PF=-3008, RL=Qmaga, #TRXS=6
Start mining: 3/4
-7216 MINE APPROVED: POA
Proposal:  18904:2.912; -3008:3.557; -6381:4.239; -1543:3.579; 69564:3.224; -7216:2.817; 
PREV =  | CURR = 18904:2.912; -3008:3.557; -6381:4.239; -1543:3.579; 69564:3.224; -7216:2.817; 
-7216 MINE APPROVED AND ACCEPTED: POQ
BLK: C:0x7fd77#  1 <-- PH=0    :PF=-7216, RL=QmXnp, #TRXS=6

5 - CONSENSUS + I

In [8]:
def predict():
    txs = []
    for t in blockchain.lastBlock.transactions:
        if t.type == 'MOD':
            txs.append(t.load)
    for c in blockchain.cli_pointers:
        model = CNNLSTM(36,2)
        weights = c.aggregate(txs)
        dev = c.predict(model, weights)
        print(c.alias, dev)

predict()

['QmVAoy7R6VTJBoLZGi1i5F16hUawzCu7QoJaWZjC8wUe12|QmShuVZ18SitU4zrvNvjMWee4SmZZhAaM8r9XtDyi1DRhM', 'QmbzRHVY41jHNuZKbM34fZPidScn74Kr22PBzJqzooq5qw|QmPsR6Ts5UjmBjiCcSqodHjQ73NshDyV18YE3CvDFhoKC5', 'QmbwvpUanJtt8b2aVtgZ7m5c42jhRUDSvuvk7V3p2oniQi|QmaxuvEpM19wRiGna8E68ZKMV3sTdiMkQ4Xz3MrUx5BH6h']


TypeError: predict() takes from 1 to 2 positional arguments but 3 were given